In [3]:
import json

objs = []
with open('../factscore_bio.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)   # parse one JSON object per line
        objs.append(obj)

print(len(objs))

683


In [5]:
sentences = [x["prompt"] for x in objs]

batch = sentences

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

In [7]:



sentences = [
    "Where was Marie Curie born?",
    "Maria Sklodowska, later known as Marie Curie, was born on November 7, 1867.",
    "Born in Paris on 15 May 1859, Pierre Curie was the son of Eugène Curie, a doctor of French Catholic origin from Alsace."
]

# Apply tokenizer
inputs = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
outputs = model(**inputs)

# Mean pooling
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings
embeddings = mean_pooling(outputs[0], inputs['attention_mask'])

In [10]:
import numpy as np

# Convert tensor outputs and embeddings to numpy arrays
outputs_array = outputs[0].detach().cpu().numpy()
embeddings_array = embeddings.detach().cpu().numpy()

# Print the dimensions (shapes) of both arrays
print("outputs array shape:", outputs_array.shape)
print("embeddings array shape:", embeddings_array.shape)


# write to file
with open('contriever_embeddings.json', 'w', encoding='utf-8') as f:
    json.dump(embeddings_array.tolist(), f)


outputs array shape: (683, 20, 768)
embeddings array shape: (683, 768)
